In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import statistics
import matplotlib.pyplot as plt

In [3]:
feature_table = pd.read_csv('data/feature_with_label_all_48H.csv')

In [11]:
patients_only = feature_table.loc[feature_table['bin_num'] == 1]

### SQL Database

In [4]:
import sqlite3 # library for working with sqlite database
conn = sqlite3.connect("/Volumes/Abbi/MIMIC.db") # Create a connection to the on-disk database

In [5]:
pd.read_sql("SELECT * FROM sqlite_master where type='table'", conn)

,type,name,tbl_name,rootpage,sql
0,table,CHARTEVENTS,CHARTEVENTS,2,"CREATE TABLE CHARTEVENTS(\n ""ROW_ID"" TEXT,\n ..."
1,table,PROCEDUREEVENTS_MV,PROCEDUREEVENTS_MV,9120960,"CREATE TABLE ""PROCEDUREEVENTS_MV"" (\n""index"" I..."
2,table,CALLOUT,CALLOUT,9132354,"CREATE TABLE ""CALLOUT"" (\n""index"" INTEGER,\n ..."
3,table,D_CPT,D_CPT,9133903,"CREATE TABLE ""D_CPT"" (\n""index"" INTEGER,\n ""R..."
4,table,D_ITEMS,D_ITEMS,9133909,"CREATE TABLE ""D_ITEMS"" (\n""index"" INTEGER,\n ..."
5,table,CAREGIVERS,CAREGIVERS,9134171,"CREATE TABLE ""CAREGIVERS"" (\n""index"" INTEGER,\..."
6,table,MICROBIOLOGYEVENTS,MICROBIOLOGYEVENTS,9134246,"CREATE TABLE ""MICROBIOLOGYEVENTS"" (\n""index"" I..."
7,table,LABEVENTS,LABEVENTS,9153054,"CREATE TABLE ""LABEVENTS"" (\n""index"" INTEGER,\n..."
8,table,INPUTEVENTS_CV,INPUTEVENTS_CV,9708019,"CREATE TABLE ""INPUTEVENTS_CV"" (\n""index"" INTEG..."
9,table,ADMISSIONS,ADMISSIONS,10291267,"CREATE TABLE ""ADMISSIONS"" (\n""index"" INTEGER,\..."


In [6]:
admissions_df = pd.read_sql("""SELECT *
                            FROM admissions
                            """, conn)

In [7]:
patients_df = pd.read_sql("""SELECT *
                            FROM patients
                            """, conn)

In [8]:
# Join admissions and patient
patient_admissions_df = pd.read_sql("""SELECT *
            FROM admissions 
            LEFT JOIN patients ON admissions.SUBJECT_ID = patients.SUBJECT_ID""", conn)

In [9]:
patient_admissions_df

,index,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,...,HAS_CHARTEVENTS_DATA,index,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,None,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,...,1,240,19,22,F,2131-05-07 00:00:00,None,None,None,0
1,1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,None,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,...,1,241,20,23,M,2082-07-17 00:00:00,None,None,None,0
2,2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,None,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,...,1,241,20,23,M,2082-07-17 00:00:00,None,None,None,0
3,3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,None,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,...,1,242,21,24,M,2100-05-31 00:00:00,None,None,None,0
4,4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,None,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,...,1,243,22,25,M,2101-11-21 00:00:00,None,None,None,0
5,5,26,26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,None,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,...,1,244,23,26,M,2054-05-04 00:00:00,2128-02-25 00:00:00,None,2128-02-25 00:00:00,1
6,6,27,27,134931,2191-11-30 22:16:00,2191-12-03 14:45:00,None,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,...,1,245,24,27,F,2191-11-30 00:00:00,None,None,None,0
7,7,28,28,162569,2177-09-01 07:15:00,2177-09-06 16:00:00,None,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,...,1,246,25,28,M,2103-04-15 00:00:00,None,None,None,0
8,8,29,30,104557,2172-10-14 14:17:00,2172-10-19 14:37:00,None,URGENT,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,...,1,247,26,30,M,1872-10-14 00:00:00,None,None,None,0
9,9,30,31,128652,2108-08-22 23:27:00,2108-08-30 15:00:00,2108-08-30 15:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,...,1,248,27,31,M,2036-05-17 00:00:00,2108-08-30 00:00:00,2108-08-30 00:00:00,2108-08-30 00:00:00,1


In [12]:
# Pull the admissions that we have in our 48 hour table
used_patients_admissions = pd.DataFrame(patient_admissions_df.loc[patient_admissions_df['HADM_ID'].isin(patients_only['HADM_ID'].values)])

In [51]:
percent_admissions_used = used_patients_admissions.shape[0]/admissions_df.shape[0] * 100
percent_admissions_used

92.14426207270753

In [16]:
# Number of patients in our 48 hour table
number_patients = used_patients_admissions['SUBJECT_ID'].iloc[:,1].value_counts().shape[0]

In [15]:
number_patients

43175

In [17]:
# Create a patient df with only the patients in our 48 hour table
used_patients = patients_df.loc[patients_df['SUBJECT_ID'].isin(used_patients_admissions['SUBJECT_ID'].iloc[:,1].values)]

In [18]:
percent_male = (used_patients['GENDER'].value_counts()[0] / number_patients) * 100
percent_male

56.18760856977417

In [42]:
# Make an age column (age at admit)
used_patients_admissions['DOB'] = pd.to_datetime(used_patients_admissions['DOB'])
used_patients_admissions['ADMITTIME'] = pd.to_datetime(used_patients_admissions['ADMITTIME'])
used_patients_admissions['AGE'] = ((used_patients_admissions['ADMITTIME'] - used_patients_admissions['DOB']).dt.days / 365)

In [24]:
used_patients_admissions[used_patients_admissions['AGE'] >= 16]['AGE'].describe()

count    45650.000000
mean        62.732506
std         16.420925
min         16.024658
25%         52.224658
50%         64.545205
75%         76.071233
max         89.060274
Name: AGE, dtype: float64

In [43]:
# Subject IDs of patients over 16
patients_16over = used_patients_admissions[used_patients_admissions['AGE'] >= 16]['SUBJECT_ID'].iloc[:,1].values

In [46]:
# HADM_IDs of patients over 16 years of age
HADM_ID_16_over = used_patients_admissions.loc[used_patients_admissions['SUBJECT_ID'].iloc[:,1].isin(patients_16over)]['HADM_ID'].values

In [47]:
# How many patients do we have that are over 16 years old?
num_patients_16_over = used_patients_admissions[used_patients_admissions['AGE'] >= 16]['SUBJECT_ID'].iloc[:,1].value_counts().shape[0]

In [48]:
num_patients_16_over

35164

In [54]:
# Take only the HADM_IDs from the 48 hour table of patients over 16
feature_table[feature_table['HADM_ID'].isin(HADM_ID_16_over)]


,Chloride_max,Creatinine_max,Glucose_max,Hematocrit_max,Hemoglobin_max,Platelets_max,Potassium_max,Red blood cell count_max,Sodium_max,White blood cell count_max,...,ethnicity_WHITE,ethnicity_WHITE - BRAZILIAN,ethnicity_WHITE - EASTERN EUROPEAN,ethnicity_WHITE - OTHER EUROPEAN,ethnicity_WHITE - RUSSIAN,diag_count,ADDITIVE_drug,BASE_drug,MAIN_drug,label
0,114.0,2.7,305.0,36.5,13.0,466.0,4.9,4.58,144.0,11.9,...,1,0,0,0,0,16,0.0,33.0,59.0,0
1,112.0,2.2,264.0,32.8,11.3,407.0,4.4,4.02,145.0,15.9,...,1,0,0,0,0,16,0.0,33.0,59.0,0
2,107.0,2.1,261.0,32.2,10.9,330.0,4.2,3.86,138.0,10.5,...,1,0,0,0,0,16,0.0,33.0,59.0,0
3,112.0,1.3,92.0,28.0,8.5,186.0,5.7,2.78,133.0,17.9,...,1,0,0,0,0,9,0.0,7.0,22.0,0
4,111.0,1.1,135.0,31.8,10.4,162.0,5.0,3.38,133.0,14.2,...,1,0,0,0,0,9,0.0,7.0,22.0,0
5,103.0,0.8,131.0,36.9,12.4,259.0,4.4,3.86,131.0,13.6,...,0,0,0,0,0,9,0.0,5.0,70.0,0
6,101.0,0.7,142.0,30.4,10.2,185.0,4.6,3.06,134.0,16.6,...,0,0,0,0,0,9,0.0,5.0,70.0,0
7,95.0,0.6,118.0,30.5,10.1,199.0,4.4,3.10,134.0,9.8,...,0,0,0,0,0,9,0.0,5.0,70.0,0
8,96.0,0.7,77.0,35.1,11.7,246.0,4.0,3.54,132.0,7.7,...,0,0,0,0,0,9,0.0,5.0,70.0,0
9,97.0,0.7,75.0,36.2,11.8,287.0,4.0,3.64,129.0,7.5,...,0,0,0,0,0,9,0.0,5.0,70.0,0
